<a href="https://colab.research.google.com/github/Minwoo-study/ELECTRA_classifier/blob/main/KcELECTRA_%EB%AA%A8%EB%8D%B8_fine_tuning_final_%EC%A1%B0%EC%84%A0%EB%8F%99%EC%95%84_%EA%B8%B0%EC%82%AC_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pytorch + HuggingFace 
## KcElectra Model
Beomi님의 KcElectra-base 사용<br>
https://github.com/Beomi/KcELECTRA<br>

## Dataset
연세 말뭉치와 위키 문헌 근대 문학에서 '저희'가 포함된 문장을 '지칭의 저희'(label 0)와 '겸양의 저희(label 1)'로 분류
만든 모델로 조선일보, 동아일보 100년 데이터에서 '저희'의 의미 분류

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
꼭 GPU로 해주세요 - 1epoch 당 약 20분 소요   
파이썬 3.9 base로 돌리기

In [ ]:
import pandas as pd
train = './train_small.csv'
test = './test_small.csv'
test_pd = pd.read_csv('./test_small.csv')

In [ ]:
test_pd

In [ ]:
#print(train.head())
#print(test.head())

# 패키지 설치

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW, TextClassificationPipeline
from tqdm.notebook import tqdm

In [ ]:
# GPU 사용
device = torch.device("cuda")

# Dataset 만들어서 불러오기 

In [ ]:
class ClassifyDataset(Dataset):
  
  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file).dropna(axis=0) 
    # 중복제거
    self.dataset.drop_duplicates(subset=['sentence'], inplace=True)

    self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base") #

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:3].values #idx 행과 0,1,2 columns
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt', #return pytorch tensors
        truncation=True, #reducing long sequences, 256개의 token만 살리고 뒤는 자름
        max_length=256,
        pad_to_max_length=False, #padding
        add_special_tokens=True #자동으로 문장 앞뒤로 special tocken - padding 부착
        )
    
    input_ids = inputs['input_ids'][0] #모델의 입력
    attention_mask = inputs['attention_mask'][0] #padding(0이면 패딩 없음)

    return input_ids, attention_mask, y

In [ ]:
train_dataset = ClassifyDataset(train)
test_dataset = ClassifyDataset(test)

# Create Model

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model, device=0)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

In [ ]:
#model.load_state_dict(torch.load("jh_model.pt"))

# Learn

In [ ]:
epochs = 5
batch_size = 16

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [ ]:
losses = []
accuracies = []

for i in range(epochs): #epoch 5
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train() #forward

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader): #tqdm 진행상황 확인 
  # train_loader batch_size = 16 -> iterations에 대해서 batches? (data size / batch size = num of iterations ---> 1 epoch)
    optimizer.zero_grad()
  
    y_batch = y_batch.type(torch.LongTensor)
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0] #to(device) : gpu에 복사본 저장(pass data to device)
    loss = F.cross_entropy(y_pred, y_batch)

    loss.backward()
    optimizer.step() #update params(weights and biases)

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1) #max로 하는 이유?
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total) #예측한 결과 loss, accuracy (지도학습)

In [ ]:
losses, accuracies

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "jh_model_shap.pt")

In [ ]:
# 모델 불러오기
model.load_state_dict(torch.load("jh_model_shap.pt"))

테스트 데이터셋 정확도 확인하기

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.type(torch.LongTensor)
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)


'''
for idx, review in enumerate(test_pd['리뷰']):
  pred = sentiment_classifier(review)
  print(f'{review}\n>> {pred[0]}')
'''

In [ ]:
#문장 하나하나 분류
def sentences_predict(sent):
    tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=256
    )
    tokenized_sent.to(device)
    
    with torch.no_grad():# 그라디엔트 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits)
    return result

In [ ]:
y_pred

In [ ]:
test_pd['sentence'].head()

#### 나름 아무 문장이나 잘 분류함

In [ ]:
sentences_predict('도대체 어쩌란 말이냐! 저희들은 그 말을 듣고 조용히 물러났다')

In [ ]:
sentences_predict('도대체 어쩌란 말이냐! "저희들이 알아서 하겠나이다"')

In [ ]:
sentences_predict('생각보다 맛있고, 사장님이 친절해요')

# 조선 동아일보 데이터 분류

In [ ]:
cd_data= pd.read_pickle('./저희_조선동아_1954_1999.pkl')
cd_data

In [ ]:
score = [] # label - score

total_len = len(cd_data)

for cnt, review in enumerate(cd_data['text']): 
  pred = sentiment_classifier(review) 
  score.append(pred) 
  print(cnt, '개 문장 분류 완료')

In [ ]:
len(score) #전체 데이터 개수와 같은지 확인

In [ ]:
cd_data['predicted'] = 0 # label(예측 결과): 1(긍정) / 0(부정)
cd_data['score']=0
 

for i in range(len(score)): 
    cd_data['predicted'][i] = int(score[i][0].get('label')[-1])
    cd_data['score'][i] = float(score[i][0].get('score'))

cd_data

In [ ]:
cd_data.to_csv('조선동아_저희 모델 분류 결과_10words.csv', index=False)
#test_pd.to_excel('저희 모델 분류 결과.xlsx', index=False)

In [ ]:
df=cd_data.groupby(['year', 'publisher', 'predicted']).count()
df

In [ ]:
df=df[['text']]
df

In [ ]:
df.to_excel('./조선동아_저희 모델 분류 결과 추세.xlsx')

### 시각화

#### 전체

In [ ]:
import os


# Mac OS의 경우와 그 외 OS의 경우로 나누어 설정

if os.name == 'posix':

    plt.rc("font", family="AppleGothic")

else :

    plt.rc("font", family="Malgun Gothic")

In [ ]:
df=cd_data.groupby(['year', 'predicted']).count()
df

In [ ]:
df.reset_index(inplace=True)
df

In [ ]:
df.set_index(['year'], inplace=True)
df

In [ ]:
df['겸양의 저희'] =0
df['지칭의 저희'] = 0

for idx in df.index :
    df['겸양의 저희'][idx] = df['text'][df['predicted']==1][idx]
    
    try :
        df['지칭의 저희'][idx] =df['text'][df['predicted']==0][idx]
    except :
    #donga['겸양의 저희'][idx] =0
        df['지칭의 저희'][idx] =0
df

In [ ]:
df= df[['겸양의 저희', '지칭의 저희']]
df.reset_index(inplace=True)
df.drop_duplicates('year', inplace=True)
df.set_index('year', inplace=True)
df.plot()

In [ ]:

donga= df[df['publisher']=='donga']

#### 조선일보

In [ ]:
chosun = df[df['publisher']=='chosun']

In [ ]:
chosun.set_index(['year'], inplace=True)
chosun

In [ ]:
chosun['겸양의 저희'] =0
chosun['지칭의 저희'] = 0

for idx in chosun.index :
    chosun['겸양의 저희'][idx] = chosun['text'][chosun['predicted']==1][idx]
    
    try :
        chosun['지칭의 저희'][idx] =chosun['text'][chosun['predicted']==0][idx]
    except :
    #donga['겸양의 저희'][idx] =0
        chosun['지칭의 저희'][idx] =0
chosun

In [ ]:
chosun= chosun[['겸양의 저희', '지칭의 저희']]

In [ ]:
len(chosun)

In [ ]:
chosun.reset_index(inplace=True)

In [ ]:
chosun

In [ ]:
chosun=chosun.drop_duplicates('year')
len(chosun)

In [ ]:
chosun.set_index('year', inplace=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
chosun.plot(title='조선일보 저희 추세 그래프')

#### 동아일보

In [ ]:
chosun = df[df['publisher']=='chosun']
donga= df[df['publisher']=='donga']

In [ ]:
donga.set_index(['year'], inplace=True)
donga

In [ ]:
donga['text'][donga['predicted']==0]

In [ ]:
donga['text'][1974]

In [ ]:
donga['겸양의 저희'] =0
donga['지칭의 저희'] = 0

for idx in donga.index :

    donga['겸양의 저희'][idx] = donga['text'][donga['predicted']==1][idx]
    try :
        donga['지칭의 저희'][idx] =donga['text'][donga['predicted']==0][idx]
    except :
    #donga['겸양의 저희'][idx] =0
        donga['지칭의 저희'][idx] =0
donga

In [ ]:
donga= donga[['겸양의 저희', '지칭의 저희']].drop_duplicates()

In [ ]:
donga

In [ ]:
fig = df.plot(title='전체 저희 추세 그래프').get_figure()
plt.tight_layout()
fig.savefig('전체 저희 추세 그래프.png')


In [ ]:
donga.plot(title='동아일보 저희 추세 그래프')

In [ ]:
chosun.plot(title='조선일보 저희 추세 그래프')